In [3]:
from copyreg import constructor
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
from evogym.envs import StepsUp, WalkingFlat
import struct
from typing import List, Tuple
import numpy as np
import dill
from graph import Graph
from evolution_strategies import tournament_selection
from population import Population
from arg_parser import parse_args
from pygifsicle import optimize
import imageio
import test

In [20]:
FILE_PATH = "results/fit_share_02.pkl"
GENERATION = "99"
ENV = "StepsUp" # "[WalkingFlat, StepsUp]"

In [21]:
with open(FILE_PATH, "rb") as file:
    structure_pop, robot_dict = dill.load(file)
    
def get_observation(env):
    a = env.get_vel_com_obs("robot")
    b = env.get_pos_com_obs("robot")
    c = env.get_floor_obs("robot", ["ground"], 5)
    return np.concatenate((a, b, c))

# def get_observation(env):
#     a = env.get_vel_com_obs("robot")
#     b = env.get_pos_com_obs("robot")
#     return np.concatenate((a, b))
    
def generate_robot(g: Graph, structure):
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            input = (i - (structure[0] // 2),
                     j - (structure[1] // 2))
            graph_out = g.operate(input)
            node = np.argmax(graph_out)
            robot[i][j] = node
    return robot

def get_best_controller(robot, controller_population, n_steps):
    robot = generate_robot(design, (5,5))    
    connections = get_full_connectivity(robot)

    
    env = StepsUp(body=robot, connections=connections)
    
    fitness = []
    cont = 0
    for controller in controller_population.indvs:
        env.reset()
        reward = test.calculate_reward(env, controller, 500)
        fitness.append((cont, reward, controller.id))
        cont += 1
        
    fitness.sort(key=lambda x: (x[1], x[0]))
    
    return fitness

In [22]:
structure_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
cont = 0
print("\t ID \t  Fitness \t Species")
for design in structure_pop.indvs:
    if design.original_fit > -999:
       
        print(f"{cont}:\t {design.id}\t| {design.original_fit:.4f}\t| {design.species_id}")
    cont+=1

	 ID 	  Fitness 	 Species
0:	 593644	| 0.0000	| 115765
1:	 597837	| 0.0000	| 115765
2:	 599694	| 0.0000	| 115765
3:	 601299	| 0.0000	| 115765
4:	 601301	| 0.0000	| 115765
5:	 601303	| 0.0000	| 122660
6:	 601305	| 0.0000	| 115765
7:	 601307	| 0.0000	| 115765
8:	 601309	| 0.0000	| 115765
9:	 601311	| 0.0000	| 115765
10:	 601313	| 0.0000	| 115765
11:	 601315	| 0.0000	| 124253
12:	 601317	| 0.0000	| 124257
13:	 601319	| 0.0000	| 115765
14:	 601321	| 0.0000	| 122660
15:	 601323	| 0.0000	| 115765
16:	 601325	| 0.0000	| 121591
17:	 601327	| 0.0000	| 124253
18:	 601329	| 0.0000	| 121591
19:	 601331	| 0.0000	| 119787
20:	 601333	| 0.0000	| 115765
21:	 601335	| 0.0000	| 115765
22:	 601337	| 0.0000	| 121591
23:	 601339	| 0.0000	| 124254
24:	 601341	| 0.0000	| 124255
25:	 601343	| 0.0000	| 115765
26:	 601345	| 0.0000	| 115765
27:	 601347	| 0.0000	| 115765
28:	 601349	| 0.0000	| 115765
29:	 601351	| 0.0000	| 124256


In [23]:
STEPS = 600
DESIGN_ID = 0
SAVE_GIF = False

In [24]:
design =structure_pop.indvs[DESIGN_ID]

robot = generate_robot(design, (5,5))
# robot = unhash("0002200024000440044400444", (5,5))
connections = get_full_connectivity(robot)
print(robot)

if ENV == "StepsUp":
    env = StepsUp(body=robot, connections=connections)
else:
    env = WalkingFlat(body=robot, connections=connections)


env.reset()
if not SAVE_GIF:
    env.render('screen')

controller_pop = robot_dict[hashable(robot)].population
controller_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
controller = controller_pop.indvs[0]
print(controller.id)

reward = 0
actuators = env.get_actuator_indices("robot")
imgs = []
for _ in range(STEPS):
    obs = get_observation(env)

    action_by_actuator = controller.operate(obs, reset_fit=False)
    action = [action_by_actuator[i] for i in actuators]
#     action = np.clip(action, .6, 1.6)
    _, r, done, _ = env.step(np.array(action))
    
    if SAVE_GIF:
        imgs.append(env.render(mode='img'))
    else:
        env.render('screen')
    reward += r

    if done:
        print("AAA")
        break

        
print(f'\ntotal reward: {round(reward, 5)}\n')
env.close()

if SAVE_GIF:
    print(len(imgs))
    imageio.mimsave(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif', imgs, duration=(1/60))

[[4. 4. 4. 0. 0.]
 [4. 4. 4. 0. 0.]
 [4. 4. 4. 0. 0.]
 [4. 4. 4. 0. 0.]
 [4. 4. 4. 0. 0.]]
478471

total reward: 4.94782



In [7]:
if SAVE_GIF:
    optimize(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif')

In [26]:
save_top = 20
ordered_robots = {k: v for k, v in sorted(robot_dict.items(), key=lambda item: -1*item[1].best_fit)}
best_robots = {}
cont = 0
for k, v in ordered_robots.items():
    if cont >= save_top:
        break
    best_robots[k] = v
    cont+=1
for indv in structure_pop.indvs:
    robot = generate_robot(indv, (5,5))
    h = hashable(robot) 
    if h not in best_robots and h in robot_dict:
        best_robots[h] = robot_dict[h]
print(len(best_robots))

29
